# Training and testing of objection detection network
Based on Faster-RCNN and this guide: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html


## Importing libraries and custom scripts


In [1]:
import sys
import os
import numpy as np
import torch
import torch.utils.data


sys.path.append(os.getcwd() + "/scripts")
import getitem as gi # Contains CansDataset class


dataset = gi.CansDataset('data/') # Does not work currently, CansDataset class needs to be fixed


## Adding pretrained model and modify numbers of classes

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


### To Do
Fix get item så den kører med Jonas' data\
Få vores data til at kører med den pretrænede model\
Test forward pass\
Benyt Holgers split til at træne med og opnå fuld model

